In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import fiona
import pydash
import shapely.geometry
from shapely.geometry import Polygon, Point, shape
import pyproj
import numpy as np
from affine import Affine
from rasterio.features import rasterize

In [ ]:
from kk.dggs import DGGS
import kk.tools
from kk import uitools
from kk.tools import save_png, dump_text, slurp

dg = DGGS()

In [ ]:
poly, src_crs = kk.tools.load_polygons('/Users/kirill/wk/dggs/data/act_aea.shp')
poly = poly[0]
display(poly, src_crs)

In [ ]:
scale_level = 12#13

overlaps = dg.compute_overlap(scale_level, *poly.boundary.xy, crs=src_crs)

if len(overlaps) != 1:
    print('Warning polygon crosses top-level DGGS regions, this is not supported yet')
    

roi = overlaps[0]
display( roi )
roi = roi.align_by(1)
display(roi)

tr, *_ = dg.pixel_coord_transform(roi, native=True, no_offset=True)


poly_m = shapely.ops.transform(dg.to_native(src_crs), poly)
display(poly_m)

poly_pix = shapely.ops.transform(lambda x,y: tr.inverse(x,y), poly_m)
display(poly_pix)

[int(b) for b in poly_pix.bounds]

In [ ]:
im = rasterize([poly_pix], out_shape=roi.shape)

ff = dg.mk_display_helper()

fig = plt.figure(figsize=(6,6))
ax = fig.add_axes((0,0,1,1))


im, extent = ff(roi.addr, im)

ax.imshow(im, extent=extent)

uitools.plot_roi(roi, 'y-', ax=ax, alpha=0.5, linewidth=2)
uitools.plot_roi(roi.scale_up(3), 'y-', ax=ax, alpha=0.6, linewidth=5)
uitools.plot_roi(roi.scale_up(4), 'y-', ax=ax, alpha=0.6, linewidth=5)
uitools.plot_roi(roi.scale_up(5), 'y-', ax=ax, alpha=0.6, linewidth=5)

In [ ]:
def to_simple_txt(oo):
    ll = []
    
    for roi, xy, *_ in oo:
        lvl = []
        for x,y in xy.T:
            addr = roi.addr + (x,y)
            lvl.append(str(addr))

        ll.extend(sorted(lvl))
    return ll


def mask_lvl_diff(m, roi, m2, m2_roi):
    assert roi.scale == m2_roi.scale + 1
    m2, m2_roi = dg.expand(m2, m2_roi)
    #import pdb; pdb.set_trace()
    m2, _ = dg.crop(m2, m2_roi, roi)
    return m*(~m2)

def mask_to_pts(m, roi, keep_diff_image=False):
    out = []
    
    while m.any():
        m2, m2_roi = dg.scale_op_and(m, roi, tight=True)
        m_diff = mask_lvl_diff(m, roi, m2, m2_roi)
        y, x = np.where(m_diff)
        
        extras = ()
        if keep_diff_image:
            extras = extras + (m_diff,)
        
        out.insert(0, (roi, np.vstack((x,y))) + extras)
        
        m, roi = m2, m2_roi
    
    return out
        
    

In [ ]:
oo = mask_to_pts(im>0, roi, keep_diff_image=True)

aa = to_simple_txt(oo)

display(len(oo), oo[0], aa[:10])

In [ ]:
def combine_masks(oo):
    dst_roi = oo[-1][0]
    out = None
    n = len(oo)
    
    for idx, (roi,_, m) in enumerate(oo):
        m = m.astype('uint8')*(n-idx)
        m, _ = dg.expand_to_roi(m, roi, dst_roi)
        
        if out is None:
            out = m
        
        assert out.shape == m.shape
        
        out[m>0] = m[m>0]
    
    return out
            
        

mm = combine_masks(oo)

palette = dict(divergent=[0xffffff,0xffffe0,0xffa474,0xdb4551,0xffffe0,0xffa474,0xdb4551,0x8b0000],
               sequential=[0xffffff, 0xffffe0,0xffd59b,0xffa474,0xf47461,0xdb4551,0xb81b34,0x8b0000])

mm_colour = kk.tools.index_to_rgb(mm, palette['sequential'], alpha=0)
plt.imshow(mm_colour)

In [ ]:
save_png('mm-mask-full-res.png', im, binary=True)
save_png('mm-colour.png', mm_colour)
dump_text(aa, 'mm-addresses.txt')
dump_text(aa, 'mm-addresses.txt.gz')
dump_text(aa, 'mm-addresses.txt.xz')

for i, (roi, xy, bm) in enumerate(oo):
    save_png(f'mm-border-{i}.png', bm.astype('uint8'), binary=True)

!ls -lh mm-*
#!open mm-colour.png

-------------------------------------------

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = fig.add_axes((0,0,1,1))
ax.set_axis_off()


for roi_, _ , a in oo:
    a,extent = ff(roi_.addr, a)
    a = a.astype('float32')
    a[a==0] = np.nan
    ax.imshow(a, extent=extent)

In [ ]:
aa_ = slurp('mm-addresses.txt.xz', lambda a: DGGS.Address(a))
aa_[:10]

In [ ]:
_roi = DGGS.roi_from_points(aa_)

display(_roi, roi)
